In [6]:
import pandas as pd
import numpy as np
import os
from matplotlib import pyplot as plt
import seaborn as sns
import sklearn
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn .model_selection import train_test_split
from sklearn.metrics import r2_score,explained_variance_score,mean_absolute_error,mean_squared_error
from math import sqrt
from datetime import datetime
%matplotlib inline

In [7]:
## converts scientific numbers to float from the import
np.set_printoptions(formatter = {"float_kind":"{:f}".format})
sns.set(rc={'figure.figsize':(12,10)})

### Load the Data

In [8]:
#load the data for total cases of corona virus infections
confirmed_cases = pd.read_csv('E:\\corona_data\\COVID-19-master\\csse_covid_19_data\\csse_covid_19_time_series\\time_series_19-covid-Confirmed.csv')
confirmed_cases.head()

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,2/25/20,2/26/20,2/27/20,2/28/20,2/29/20,3/1/20,3/2/20,3/3/20,3/4/20,3/5/20
0,Anhui,Mainland China,31.8257,117.2264,1,9,15,39,60,70,...,989,989,989,990,990,990,990,990,990,990
1,Beijing,Mainland China,40.1824,116.4142,14,22,36,41,68,80,...,400,400,410,410,411,413,414,414,418,418
2,Chongqing,Mainland China,30.0572,107.8740,6,9,27,57,75,110,...,576,576,576,576,576,576,576,576,576,576
3,Fujian,Mainland China,26.0789,117.9874,1,5,10,18,35,59,...,294,294,296,296,296,296,296,296,296,296
4,Gansu,Mainland China,36.0611,103.8343,0,2,2,4,7,14,...,91,91,91,91,91,91,91,91,91,102


In [10]:
#load the data for the total confirmed deaths
confirmed_deaths = pd.read_csv('E:\\corona_data\\COVID-19-master\\csse_covid_19_data\\csse_covid_19_time_series\\time_series_19-covid-Deaths.csv')
confirmed_deaths.head()

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,2/25/20,2/26/20,2/27/20,2/28/20,2/29/20,3/1/20,3/2/20,3/3/20,3/4/20,3/5/20
0,Anhui,Mainland China,31.8257,117.2264,0,0,0,0,0,0,...,6,6,6,6,6,6,6,6,6,6
1,Beijing,Mainland China,40.1824,116.4142,0,0,0,0,0,1,...,4,4,5,7,8,8,8,8,8,8
2,Chongqing,Mainland China,30.0572,107.8740,0,0,0,0,0,0,...,6,6,6,6,6,6,6,6,6,6
3,Fujian,Mainland China,26.0789,117.9874,0,0,0,0,0,0,...,1,1,1,1,1,1,1,1,1,1
4,Gansu,Mainland China,36.0611,103.8343,0,0,0,0,0,0,...,2,2,2,2,2,2,2,2,2,2


In [11]:
#load the data for total recoveries
confirmed_recoveries = pd.read_csv('E:\\corona_data\\COVID-19-master\\csse_covid_19_data\\csse_covid_19_time_series\\time_series_19-covid-Recovered.csv')
confirmed_recoveries.head()

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,2/25/20,2/26/20,2/27/20,2/28/20,2/29/20,3/1/20,3/2/20,3/3/20,3/4/20,3/5/20
0,Anhui,Mainland China,31.8257,117.2264,0,0,0,0,0,0,...,712,744,792,821,868,873,917,936,956,970
1,Beijing,Mainland China,40.1824,116.4142,0,0,1,2,2,2,...,215,235,248,257,271,276,282,288,297,297
2,Chongqing,Mainland China,30.0572,107.8740,0,0,0,0,0,0,...,372,384,401,422,438,450,469,490,502,512
3,Fujian,Mainland China,26.0789,117.9874,0,0,0,0,0,0,...,199,218,228,235,243,247,255,260,270,277
4,Gansu,Mainland China,36.0611,103.8343,0,0,0,0,0,0,...,80,81,81,82,82,84,85,86,87,87


In [12]:
#Unpivot the data
Total_confirmed_infections = pd.melt(confirmed_cases,id_vars=['Province/State','Country/Region','Lat','Long'],var_name=['Date'])

In [13]:
Total_confirmed_deaths = pd.melt(confirmed_deaths,id_vars=['Province/State','Country/Region','Lat','Long'],var_name=['Date'])

In [14]:
Total_confirmed_recoveries = pd.melt(confirmed_recoveries,id_vars=['Province/State','Country/Region','Lat','Long'],var_name=['Date'])

In [15]:
print("The shape of confirmed is :", Total_confirmed_infections.shape)
print("The shape of confirmed is :", Total_confirmed_deaths.shape)
print("The shape of confirmed is :", Total_confirmed_recoveries.shape)

The shape of confirmed is : (7612, 6)
The shape of confirmed is : (7612, 6)
The shape of confirmed is : (7612, 6)


In [16]:
Total_confirmed_infections.head()

,Province/State,Country/Region,Lat,Long,Date,value
0,Anhui,Mainland China,31.8257,117.2264,1/22/20,1
1,Beijing,Mainland China,40.1824,116.4142,1/22/20,14
2,Chongqing,Mainland China,30.0572,107.8740,1/22/20,6
3,Fujian,Mainland China,26.0789,117.9874,1/22/20,1
4,Gansu,Mainland China,36.0611,103.8343,1/22/20,0


In [17]:
Total_confirmed_deaths.head()

,Province/State,Country/Region,Lat,Long,Date,value
0,Anhui,Mainland China,31.8257,117.2264,1/22/20,0
1,Beijing,Mainland China,40.1824,116.4142,1/22/20,0
2,Chongqing,Mainland China,30.0572,107.8740,1/22/20,0
3,Fujian,Mainland China,26.0789,117.9874,1/22/20,0
4,Gansu,Mainland China,36.0611,103.8343,1/22/20,0


In [18]:
# convert the date column to a Date Value
Total_confirmed_infections['Date'] = pd.to_datetime(Total_confirmed_infections['Date'])
Total_confirmed_deaths['Date'] = pd.to_datetime(Total_confirmed_deaths['Date'])
Total_confirmed_recoveries['Date'] = pd.to_datetime(Total_confirmed_recoveries['Date'])


In [19]:
#rename the values
Total_confirmed_infections.columns= Total_confirmed_infections.columns.str.replace('value','confirmed')
Total_confirmed_infections.head()

,Province/State,Country/Region,Lat,Long,Date,confirmed
0,Anhui,Mainland China,31.8257,117.2264,2020-01-22,1
1,Beijing,Mainland China,40.1824,116.4142,2020-01-22,14
2,Chongqing,Mainland China,30.0572,107.8740,2020-01-22,6
3,Fujian,Mainland China,26.0789,117.9874,2020-01-22,1
4,Gansu,Mainland China,36.0611,103.8343,2020-01-22,0


In [20]:
Total_confirmed_deaths.columns= Total_confirmed_deaths.columns.str.replace('value','Deaths')
Total_confirmed_deaths.head()

,Province/State,Country/Region,Lat,Long,Date,Deaths
0,Anhui,Mainland China,31.8257,117.2264,2020-01-22,0
1,Beijing,Mainland China,40.1824,116.4142,2020-01-22,0
2,Chongqing,Mainland China,30.0572,107.8740,2020-01-22,0
3,Fujian,Mainland China,26.0789,117.9874,2020-01-22,0
4,Gansu,Mainland China,36.0611,103.8343,2020-01-22,0


In [21]:
Total_confirmed_recoveries.columns= Total_confirmed_recoveries.columns.str.replace('value','Recovered')
Total_confirmed_recovered = Total_confirmed_recoveries.rename(columns= {'Deaths': 'Recovered'})
Total_confirmed_recovered.head()


,Province/State,Country/Region,Lat,Long,Date,Recovered
0,Anhui,Mainland China,31.8257,117.2264,2020-01-22,0
1,Beijing,Mainland China,40.1824,116.4142,2020-01-22,0
2,Chongqing,Mainland China,30.0572,107.8740,2020-01-22,0
3,Fujian,Mainland China,26.0789,117.9874,2020-01-22,0
4,Gansu,Mainland China,36.0611,103.8343,2020-01-22,0


#### Check for Null Values in the data

In [22]:
Total_confirmed_infections.isnull().sum()

Province/State    3608
Country/Region       0
Lat                  0
Long                 0
Date                 0
confirmed            0
dtype: int64

#### Deal with the null values by adding NA

In [23]:
Total_confirmed_infections['Province/State'].fillna(Total_confirmed_infections['Country/Region'],inplace=True)
Total_confirmed_deaths['Province/State'].fillna(Total_confirmed_infections['Country/Region'],inplace=True)
Total_confirmed_recovered['Province/State'].fillna(Total_confirmed_infections['Country/Region'],inplace=True)

Total_confirmed_infections.isnull().sum()

Province/State    0
Country/Region    0
Lat               0
Long              0
Date              0
confirmed         0
dtype: int64

In [24]:
print("The Shape of Cornirmed is: ", Total_confirmed_infections.shape)
print("The Shape of Cornirmed is: ", Total_confirmed_deaths.shape)
print("The Shape of Cornirmed is: ", Total_confirmed_recovered.shape)

The Shape of Cornirmed is:  (7612, 6)
The Shape of Cornirmed is:  (7612, 6)
The Shape of Cornirmed is:  (7612, 6)


### Join the Three Data Sets

In [25]:
#full joins
#confirmed_infections with confirmed_deaths
full_join = Total_confirmed_infections.merge(Total_confirmed_deaths[['Province/State','Country/Region','Date','Deaths']],
                                            how = 'outer',
                                            left_on = ['Province/State','Country/Region','Date'],
                                            right_on = ['Province/State','Country/Region','Date'])
print('Shape of first join: ',full_join.shape)

full_join = full_join.merge(Total_confirmed_recovered[['Province/State','Country/Region','Date','Recovered']],
                           how = 'outer',
                           left_on = ['Province/State','Country/Region','Date'],
                           right_on = ['Province/State','Country/Region','Date'])

print('Shape of first join: ',full_join.shape)
full_join.head()

Shape of first join:  (7612, 7)
Shape of first join:  (7612, 8)


,Province/State,Country/Region,Lat,Long,Date,confirmed,Deaths,Recovered
0,Anhui,Mainland China,31.8257,117.2264,2020-01-22,1,0,0
1,Beijing,Mainland China,40.1824,116.4142,2020-01-22,14,0,0
2,Chongqing,Mainland China,30.0572,107.8740,2020-01-22,6,0,0
3,Fujian,Mainland China,26.0789,117.9874,2020-01-22,1,0,0
4,Gansu,Mainland China,36.0611,103.8343,2020-01-22,0,0,0


In [26]:
full_join.isnull().sum()

Province/State    0
Country/Region    0
Lat               0
Long              0
Date              0
confirmed         0
Deaths            0
Recovered         0
dtype: int64

In [27]:
### Add month and year as a new column
full_join['Month-Year'] = full_join['Date'].dt.strftime('%b-%Y')


In [28]:
full_join.head()

,Province/State,Country/Region,Lat,Long,Date,confirmed,Deaths,Recovered,Month-Year
0,Anhui,Mainland China,31.8257,117.2264,2020-01-22,1,0,0,Jan-2020
1,Beijing,Mainland China,40.1824,116.4142,2020-01-22,14,0,0,Jan-2020
2,Chongqing,Mainland China,30.0572,107.8740,2020-01-22,6,0,0,Jan-2020
3,Fujian,Mainland China,26.0789,117.9874,2020-01-22,1,0,0,Jan-2020
4,Gansu,Mainland China,36.0611,103.8343,2020-01-22,0,0,0,Jan-2020


#### Get the Daily report by the numbers

In [32]:
# we are goint to filter the data and just use for Anhui Province.

test = full_join[full_join['Province/State']=='Anhui']

# create another dataframe with the same info as above.

full_join2 = test.copy()

In [33]:
#create a new date column less one day
full_join2['Date-1'] = full_join2['Date'] + pd.Timedelta(days=1)
full_join2.rename(columns={'confirmed':'Confirmed-1','Deaths':'Deaths-1','Recovered':'Recovered-1','Date':'Date Minus 1'},inplace=True)

In [34]:
# Join the two Data Frames
full_join3= test.merge(full_join2[['Province/State','Country/Region','Confirmed-1','Deaths-1','Recovered-1',
                                   'Date-1','Date Minus 1']], how = 'outer', 
                        left_on = ['Province/State','Country/Region','Date'],
                        right_on = ['Province/State','Country/Region','Date-1'])
                                                                                        
                       

In [35]:
full_join3['Confirmed Daily'] = full_join3['confirmed'] - full_join3['Confirmed-1']

In [37]:
test.head()

,Province/State,Country/Region,Lat,Long,Date,confirmed,Deaths,Recovered,Month-Year
0,Anhui,Mainland China,31.8257,117.2264,2020-01-22,1,0,0,Jan-2020
173,Anhui,Mainland China,31.8257,117.2264,2020-01-23,9,0,0,Jan-2020
346,Anhui,Mainland China,31.8257,117.2264,2020-01-24,15,0,0,Jan-2020
519,Anhui,Mainland China,31.8257,117.2264,2020-01-25,39,0,0,Jan-2020
692,Anhui,Mainland China,31.8257,117.2264,2020-01-26,60,0,0,Jan-2020


In [38]:
full_join2.head()

,Province/State,Country/Region,Lat,Long,Date Minus 1,Confirmed-1,Deaths-1,Recovered-1,Month-Year,Date-1
0,Anhui,Mainland China,31.8257,117.2264,2020-01-22,1,0,0,Jan-2020,2020-01-23
173,Anhui,Mainland China,31.8257,117.2264,2020-01-23,9,0,0,Jan-2020,2020-01-24
346,Anhui,Mainland China,31.8257,117.2264,2020-01-24,15,0,0,Jan-2020,2020-01-25
519,Anhui,Mainland China,31.8257,117.2264,2020-01-25,39,0,0,Jan-2020,2020-01-26
692,Anhui,Mainland China,31.8257,117.2264,2020-01-26,60,0,0,Jan-2020,2020-01-27


In [39]:
full_join3.head()

,Province/State,Country/Region,Lat,Long,Date,confirmed,Deaths,Recovered,Month-Year,Confirmed-1,Deaths-1,Recovered-1,Date-1,Date Minus 1,Confirmed Daily
0,Anhui,Mainland China,31.8257,117.2264,2020-01-22,1.0,0.0,0.0,Jan-2020,NaN,NaN,NaN,NaT,NaT,NaN
1,Anhui,Mainland China,31.8257,117.2264,2020-01-23,9.0,0.0,0.0,Jan-2020,1.0,0.0,0.0,2020-01-23,2020-01-22,8.0
2,Anhui,Mainland China,31.8257,117.2264,2020-01-24,15.0,0.0,0.0,Jan-2020,9.0,0.0,0.0,2020-01-24,2020-01-23,6.0
3,Anhui,Mainland China,31.8257,117.2264,2020-01-25,39.0,0.0,0.0,Jan-2020,15.0,0.0,0.0,2020-01-25,2020-01-24,24.0
4,Anhui,Mainland China,31.8257,117.2264,2020-01-26,60.0,0.0,0.0,Jan-2020,39.0,0.0,0.0,2020-01-26,2020-01-25,21.0


In [40]:
## Since te code has worked for one State we can apply it to all States.
#create a new data frame
full_join2 = full_join.copy()

#create a new date column less one.
full_join2['Date-1'] = full_join2['Date'] + pd.Timedelta(days=1)
full_join2.rename(columns={'confirmed':'Confirmed-1','Deaths':'Deaths-1','Recovered':'Recovered-1','Date':'Date Minus 1'},inplace=True)

In [42]:
#Join the two dataFrames
full_join3= full_join.merge(full_join2[['Province/State','Country/Region','Confirmed-1','Deaths-1','Recovered-1',
                                   'Date-1','Date Minus 1']], how = 'left', 
                        left_on = ['Province/State','Country/Region','Date'],
                        right_on = ['Province/State','Country/Region','Date-1'])
full_join3.head()
                                                                                   

,Province/State,Country/Region,Lat,Long,Date,confirmed,Deaths,Recovered,Month-Year,Confirmed-1,Deaths-1,Recovered-1,Date-1,Date Minus 1
0,Anhui,Mainland China,31.8257,117.2264,2020-01-22,1,0,0,Jan-2020,NaN,NaN,NaN,NaT,NaT
1,Beijing,Mainland China,40.1824,116.4142,2020-01-22,14,0,0,Jan-2020,NaN,NaN,NaN,NaT,NaT
2,Chongqing,Mainland China,30.0572,107.8740,2020-01-22,6,0,0,Jan-2020,NaN,NaN,NaN,NaT,NaT
3,Fujian,Mainland China,26.0789,117.9874,2020-01-22,1,0,0,Jan-2020,NaN,NaN,NaN,NaT,NaT
4,Gansu,Mainland China,36.0611,103.8343,2020-01-22,0,0,0,Jan-2020,NaN,NaN,NaN,NaT,NaT


In [43]:
full_join3['Confirmed Daily'] = full_join3['confirmed'] - full_join3['Confirmed-1']
full_join3['Deaths Daily'] = full_join3['Deaths'] - full_join3['Deaths-1']
full_join3['Recovered Daily'] = full_join3['Recovered'] - full_join3['Recovered-1']

In [44]:
print(full_join3.shape)

(7612, 17)


In [45]:
full_join3.head()

,Province/State,Country/Region,Lat,Long,Date,confirmed,Deaths,Recovered,Month-Year,Confirmed-1,Deaths-1,Recovered-1,Date-1,Date Minus 1,Confirmed Daily,Deaths Daily,Recovered Daily
0,Anhui,Mainland China,31.8257,117.2264,2020-01-22,1,0,0,Jan-2020,NaN,NaN,NaN,NaT,NaT,NaN,NaN,NaN
1,Beijing,Mainland China,40.1824,116.4142,2020-01-22,14,0,0,Jan-2020,NaN,NaN,NaN,NaT,NaT,NaN,NaN,NaN
2,Chongqing,Mainland China,30.0572,107.8740,2020-01-22,6,0,0,Jan-2020,NaN,NaN,NaN,NaT,NaT,NaN,NaN,NaN
3,Fujian,Mainland China,26.0789,117.9874,2020-01-22,1,0,0,Jan-2020,NaN,NaN,NaN,NaT,NaT,NaN,NaN,NaN
4,Gansu,Mainland China,36.0611,103.8343,2020-01-22,0,0,0,Jan-2020,NaN,NaN,NaN,NaT,NaT,NaN,NaN,NaN


In [46]:
# As seen the first day numbers are empty so we have to add them manually
full_join3['Confirmed Daily'].loc[full_join3['Date'] == '2020-01-22'] = full_join3['confirmed']
full_join3['Deaths Daily'].loc[full_join3['Date']== '2020-01-22']= full_join3['Deaths']
full_join3['Recovered Daily'].loc[full_join3['Date']== '2020-01-22']=full_join3['Recovered']

C:\Users\Gachengoh\Anaconda3\envs\PythonGPU\lib\site-packages\pandas\core\indexing.py:205: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


In [48]:
# deleting columns
del full_join3['Confirmed-1']
del full_join3['Deaths-1']
del full_join3['Recovered-1']
del full_join3['Date-1']
del full_join3['Date Minus 1']

In [49]:
full_join3.head()

,Province/State,Country/Region,Lat,Long,Date,confirmed,Deaths,Recovered,Month-Year,Confirmed Daily,Deaths Daily,Recovered Daily
0,Anhui,Mainland China,31.8257,117.2264,2020-01-22,1,0,0,Jan-2020,1.0,0.0,0.0
1,Beijing,Mainland China,40.1824,116.4142,2020-01-22,14,0,0,Jan-2020,14.0,0.0,0.0
2,Chongqing,Mainland China,30.0572,107.8740,2020-01-22,6,0,0,Jan-2020,6.0,0.0,0.0
3,Fujian,Mainland China,26.0789,117.9874,2020-01-22,1,0,0,Jan-2020,1.0,0.0,0.0
4,Gansu,Mainland China,36.0611,103.8343,2020-01-22,0,0,0,Jan-2020,0.0,0.0,0.0


In [51]:
#Export the data
full_join3.to_csv('E:\\corona_data\\CoronaVirusData.csv')